<a href="https://colab.research.google.com/github/DIWEERAPURA/Nature-Inspired-Computing-Algorithms---Real-world-Usage/blob/main/ACO_Exam_Time_Tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Google Colab–ready Advanced ACO for Exam Timetabling

import numpy as np
import random

# ---------------------------
# Setup and Parameterization
# ---------------------------

# For reproducibility in Colab:
np.random.seed(42)
random.seed(42)

# Problem parameters
NUM_EXAMS = 10
NUM_TIMESLOTS = 5
NUM_ROOMS = 3
ROOM_CAPACITY = [30, 50, 40]         # Capacities for each room
NUM_PROFESSORS = NUM_EXAMS           # One professor per exam

# ACO parameters
NUM_ANTS = 20
NUM_ITERATIONS = 100
PHEROMONE_INIT = 1.0
EVAPORATION_RATE = 0.5
ALPHA = 1.0                        # Importance of pheromone
BETA = 2.0                         # Importance of heuristic info
ELITISM_FACTOR = 2.0               # Extra reinforcement for global best

# ---------------------------
# Input Data and Precomputation
# ---------------------------

# Random binary matrix: 50 students and their exam enrollments (0: not taking, 1: taking)
STUDENT_EXAM_MATRIX = np.random.randint(0, 2, (50, NUM_EXAMS))

# Random binary matrix for professor availability: each professor (per exam) over available timeslots
PROFESSOR_AVAILABILITY = np.random.randint(0, 2, (NUM_PROFESSORS, NUM_TIMESLOTS))

# Dictionary specifying preferred timeslots for some exams/professors.
# (If an exam’s timeslot is preferred, the heuristic gets a bonus.)
PREFERRED_TIMES = {0: [0, 1], 1: [2], 2: [1, 3]}

# Room adjacency matrix (can be used for further extensions)
ROOM_ADJACENCY = np.random.rand(NUM_ROOMS, NUM_ROOMS)

# Precompute the conflict matrix:
# conflict_matrix[i, j] = number of students taking both exam i and exam j.
conflict_matrix = np.dot(STUDENT_EXAM_MATRIX.T, STUDENT_EXAM_MATRIX)
np.fill_diagonal(conflict_matrix, 0)
# Compute per-exam conflict penalty (sum over conflicts with other exams)
conflict_penalties = np.sum(conflict_matrix, axis=1)

# Initialize pheromone matrix with dimensions: (exams x timeslots x rooms)
pheromone = np.ones((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS)) * PHEROMONE_INIT

# ---------------------------
# Heuristic Function
# ---------------------------

def calculate_heuristic_matrix():
    """
    Compute a heuristic value for each (exam, timeslot, room) based on:
      - Professor availability (binary: 0 or 1)
      - Preferred timeslot bonus (1 if preferred, else 0.5)
      - Room capacity (higher capacity is favorable)
      - Precomputed conflict penalty (more conflicts lower the heuristic)
    """
    heuristic = np.zeros((NUM_EXAMS, NUM_TIMESLOTS, NUM_ROOMS))
    for exam in range(NUM_EXAMS):
        for timeslot in range(NUM_TIMESLOTS):
            availability = PROFESSOR_AVAILABILITY[exam, timeslot]
            # Apply bonus if the timeslot is preferred for this exam; else, a lower base value.
            pref_bonus = 1.0 if timeslot in PREFERRED_TIMES.get(exam, []) else 0.5
            for room in range(NUM_ROOMS):
                capacity = ROOM_CAPACITY[room]
                # The heuristic favors higher availability, preferred times, and larger capacity,
                # while penalizing exams with many conflicts.
                heuristic[exam, timeslot, room] = (availability * pref_bonus * capacity) / (1 + conflict_penalties[exam])
    return heuristic

# ---------------------------
# Fitness Evaluation
# ---------------------------

def evaluate_solution(solution):
    """
    Evaluate the quality of a timetabling solution.
    Penalties include:
      - Student exam overlaps (if a student is scheduled for multiple exams in the same timeslot)
      - Assignments where the professor is unavailable
      - Non-preferred timeslot assignment
    Lower penalty values indicate better solutions.
    """
    penalty = 0

    # Student overlap penalty: count duplicate timeslot assignments for each student.
    for student in range(STUDENT_EXAM_MATRIX.shape[0]):
        # Get exams that the student is taking.
        exam_indices = np.where(STUDENT_EXAM_MATRIX[student] == 1)[0]
        # Retrieve assigned timeslots for those exams.
        assigned_times = [solution[exam][0] for exam in exam_indices]
        # Each duplicate in timeslot assignment contributes to the penalty.
        penalty += len(assigned_times) - len(set(assigned_times))

    # Penalty for professor unavailability and non-preferred timeslot.
    for exam, (timeslot, room) in solution.items():
        if not PROFESSOR_AVAILABILITY[exam, timeslot]:
            penalty += 10   # Heavy penalty for scheduling when professor is unavailable
        if timeslot not in PREFERRED_TIMES.get(exam, []):
            penalty += 2    # Penalty for non-preferred timeslot

    return penalty

# ---------------------------
# Advanced ACO Algorithm
# ---------------------------

def aco_timetabling():
    global pheromone
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(NUM_ITERATIONS):
        solutions = []
        fitness_scores = []

        # Compute heuristic matrix once per iteration (it remains static in this model)
        heuristic = calculate_heuristic_matrix()

        # Each ant constructs a complete solution.
        for ant in range(NUM_ANTS):
            solution = {}
            for exam in range(NUM_EXAMS):
                # Build the probability matrix for each (timeslot, room) combination.
                prob_matrix = np.zeros((NUM_TIMESLOTS, NUM_ROOMS))
                for timeslot in range(NUM_TIMESLOTS):
                    for room in range(NUM_ROOMS):
                        prob_matrix[timeslot, room] = (pheromone[exam, timeslot, room] ** ALPHA) * (heuristic[exam, timeslot, room] ** BETA)

                # Normalize the probability distribution (with a safeguard for zero sum)
                total = np.sum(prob_matrix)
                if total == 0:
                    probabilities = np.full(prob_matrix.shape, 1/(NUM_TIMESLOTS * NUM_ROOMS))
                else:
                    probabilities = prob_matrix / total

                # Choose one timeslot-room combination based on the computed probabilities.
                flat_index = np.random.choice(np.arange(NUM_TIMESLOTS * NUM_ROOMS), p=probabilities.flatten())
                chosen_timeslot, chosen_room = np.unravel_index(flat_index, (NUM_TIMESLOTS, NUM_ROOMS))
                solution[exam] = (chosen_timeslot, chosen_room)

            fitness = evaluate_solution(solution)
            solutions.append(solution)
            fitness_scores.append(fitness)

        # ---------------------------
        # Pheromone Update
        # ---------------------------

        # Evaporation step reduces all pheromone levels.
        pheromone *= (1 - EVAPORATION_RATE)

        # Deposit pheromone based on each ant's solution quality.
        for sol, fit in zip(solutions, fitness_scores):
            for exam, (timeslot, room) in sol.items():
                pheromone[exam, timeslot, room] += 1.0 / (fit + 1e-6)

        # Identify the best solution in the current iteration.
        current_best_fit = min(fitness_scores)
        if current_best_fit < best_fitness:
            best_fitness = current_best_fit
            best_solution = solutions[fitness_scores.index(current_best_fit)]

        # Elitist update: extra reinforcement for the globally best solution.
        for exam, (timeslot, room) in best_solution.items():
            pheromone[exam, timeslot, room] += ELITISM_FACTOR / (best_fitness + 1e-6)

        # Display progress.
        print(f"Iteration {iteration + 1}/{NUM_ITERATIONS}: Best Fitness = {best_fitness}")

    return best_solution, best_fitness

# ---------------------------
# Run the Algorithm and Display Results
# ---------------------------

# In a Colab cell, simply run the code to see the results.
best_solution, best_fitness = aco_timetabling()
print("\n=== Final Best Timetabling Solution ===")
for exam in range(NUM_EXAMS):
    timeslot, room = best_solution[exam]
    print(f"Exam {exam}: Timeslot {timeslot}, Room {room}")
print("Best Fitness:", best_fitness)


Iteration 1/100: Best Fitness = 86
Iteration 2/100: Best Fitness = 85
Iteration 3/100: Best Fitness = 85
Iteration 4/100: Best Fitness = 85
Iteration 5/100: Best Fitness = 85
Iteration 6/100: Best Fitness = 85
Iteration 7/100: Best Fitness = 85
Iteration 8/100: Best Fitness = 85
Iteration 9/100: Best Fitness = 85
Iteration 10/100: Best Fitness = 80
Iteration 11/100: Best Fitness = 79
Iteration 12/100: Best Fitness = 79
Iteration 13/100: Best Fitness = 79
Iteration 14/100: Best Fitness = 79
Iteration 15/100: Best Fitness = 79
Iteration 16/100: Best Fitness = 79
Iteration 17/100: Best Fitness = 79
Iteration 18/100: Best Fitness = 79
Iteration 19/100: Best Fitness = 79
Iteration 20/100: Best Fitness = 79
Iteration 21/100: Best Fitness = 79
Iteration 22/100: Best Fitness = 79
Iteration 23/100: Best Fitness = 79
Iteration 24/100: Best Fitness = 79
Iteration 25/100: Best Fitness = 79
Iteration 26/100: Best Fitness = 79
Iteration 27/100: Best Fitness = 79
Iteration 28/100: Best Fitness = 79
I